

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import pandas as pd
import requests

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [3]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

input

In [4]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
190,4.930570,52.353734,4.886966,52.367779,2020-10-29 14:35:36
8,4.826993,52.365339,4.874426,52.363849,2020-10-29 15:20:31
853,4.891115,52.360369,4.880962,52.369125,2020-10-29 15:24:50
279,4.870277,52.355903,4.862036,52.367262,2020-10-29 15:09:19
267,4.901919,52.359926,4.893679,52.354495,2020-10-29 14:59:37


sample query (before you can query the server you need to run it -see run OTP server.ipynb in this folder)

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [5]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 608,
 'walkDistance': 228.47644360329696,
 'transfers': 0,
 'transitTime': 402,
 'waitingTime': 2,
 'modes': [['WALK', 159, 173], ['TRAM', 402, 1318], ['WALK', 45, 54]]}

#### visualization

In [6]:
plot(r.json()['plan'], color = 'green')

Trip from (4.8493,52.3699) to (4.8706,52.3711) at 1603979160000. 
3 connections found. 
Best one is 10min (228m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	174	159
2	TRAM	1319	402
3	WALK	54	45


output

In [7]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

FileNotFoundError: [Errno 2] File georequests_PT.csv does not exist: 'georequests_PT.csv'

full response

In [ ]:
r.json()

---